In [3]:
import sys
sys.path.insert(1, '/home/eatitall_scripts')
sys.path.insert(1, '/home/root/pctobs/lib/python3.8/site-packages')
import pandas as pd
import json
import re

def encontrar_alimentos(df,vocab_alimentos):
    todos_los_alimentos = [item for sublist in vocab_alimentos.values() for item in sublist]
    for k in range(0,len(df)):
        texto_entrada=df['observaciones'][k]
        texto = texto_entrada.lower()
        palabras = re.split(r'[ ,.;]+', texto)
        alimentos_encontrados = []
        i = 0
        while i < len(palabras):
            max_longitud = 0
            alimento_a_agregar = ''
            for alimento in todos_los_alimentos:
                alimento_partes = re.split(r'[ ,.;]+',alimento.lower())
                longitud = len(alimento_partes)
                # Comprobar si la secuencia de palabras coincide con algún alimento
                if palabras[i:i+longitud] == alimento_partes and longitud > max_longitud:
                    # Guardar el alimento más largo que coincide
                    alimento_a_agregar = ' '.join(palabras[i:i+longitud])
                    max_longitud = longitud
            if max_longitud > 0:
                alimentos_encontrados.append(alimento_a_agregar)
                i += max_longitud  # Ajustar el índice según la longitud del alimento encontrado más largo
            else:
                i += 1
        alimentos_string = ', '.join(alimentos_encontrados)
        df.loc[k,'alimentos_encontrados']=alimentos_string
    return df

In [11]:
import sys
import pandas as pd
import json
import re

# Insertando rutas al sys.path
sys.path.insert(1, '/home/eatitall_scripts')
sys.path.insert(1, '/home/root/pctobs/lib/python3.8/site-packages')

# Función para calcular la distancia de Levenshtein
def distancia_levenshtein(s1, s2):
    if len(s1) < len(s2):
        return distancia_levenshtein(s2, s1)

    if len(s2) == 0:
        return len(s1)

    matriz_previa = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        matriz_actual = [i + 1]
        for j, c2 in enumerate(s2):
            inserciones = matriz_previa[j + 1] + 1
            eliminaciones = matriz_actual[j] + 1
            sustituciones = matriz_previa[j] + (c1 != c2)
            matriz_actual.append(min(inserciones, eliminaciones, sustituciones))
        matriz_previa = matriz_actual
    
    return matriz_previa[-1]

# Función mejorada para encontrar alimentos considerando la distancia de Levenshtein
def encontrar_alimentos(df, vocab_alimentos, umbral_levenshtein=2):
    todos_los_alimentos = [item for sublist in vocab_alimentos.values() for item in sublist]
    for k in range(len(df)):
        texto_entrada = df['observaciones'][k].lower()
        palabras = re.split(r'[ ,.;]+', texto_entrada)
        alimentos_encontrados = []

        i = 0
        while i < len(palabras):
            max_longitud = 0
            alimento_a_agregar = ''
            for alimento in todos_los_alimentos:
                alimento_partes = re.split(r'[ ,.;]+', alimento.lower())
                longitud = len(alimento_partes)
                secuencia_palabras = ' '.join(palabras[i:i+longitud])
                # Calcular la distancia de Levenshtein para cada parte del alimento
                distancia = sum(distancia_levenshtein(p, ap) for p, ap in zip(palabras[i:i+longitud], alimento_partes)) / longitud
                # Verificar si la distancia está dentro del umbral y es la coincidencia más larga
                if distancia <= umbral_levenshtein and longitud > max_longitud:
                    alimento_a_agregar = secuencia_palabras
                    max_longitud = longitud
            if max_longitud > 0:
                alimentos_encontrados.append(alimento_a_agregar)
                i += max_longitud
            else:
                i += 1

        alimentos_string = ', '.join(alimentos_encontrados)
        df.loc[k, 'alimentos_encontrados'] = alimentos_string
    return df


In [7]:
distancia_levenshtein("hola que tal","hola tal")

4

In [8]:
import unicodedata

def eliminar_acentos(texto):
    """
    Elimina los acentos del texto proporcionado.
    :param texto: cadena de texto a la que se le quieren eliminar los acentos.
    :return: texto sin acentos.
    """
    texto_sin_acentos = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')
    return texto_sin_acentos

# Ejemplo de uso:
texto_con_acentos = "Esto es un texto con ácentos: á, é, í, ó, ú, ñ."
texto_sin_acentos = eliminar_acentos(texto_con_acentos)
print(texto_sin_acentos)


Esto es un texto con acentos: a, e, i, o, u, n.


In [42]:
def nerc_diccionario(df,diccionario,tipo_entidad):
    if tipo_entidad=="alimentos":
        todos_los_elementos = [item for sublist in diccionario.values() for item in sublist]
    if tipo_entidad=="farmacos" or tipo_entidad=="sintomas" or tipo_entidad=="pruebas clinicas":
        todos_los_elementos = [elemento['nombre'] for sublist in diccionario.values() for elemento in sublist]
    for k in range(0,len(df)):
        texto_entrada=df['observaciones'][k]
        texto = texto_entrada.lower()
        texto_entrada=unicodedata.normalize('NFKD', texto_entrada).encode('ASCII', 'ignore').decode('ASCII') #Eliminar acentos
        palabras = re.split(r'[ ,.;]+', texto)
        elementos_encontrados = []
        i = 0
        while i < len(palabras):
            max_longitud = 0
            elemento_a_agregar = ''
            for elemento in todos_los_elementos:
                elemento_partes = re.split(r'[ ,.;]+',elemento.lower())
                longitud = len(elemento_partes)
                # Comprobar si la secuencia de palabras coincide con algún alimento
                if palabras[i:i+longitud] == elemento_partes and longitud > max_longitud:
                    # Guardar el alimento más largo que coincide
                    elemento_a_agregar = ' '.join(palabras[i:i+longitud])
                    max_longitud = longitud
            if max_longitud > 0:
                elementos_encontrados.append(elemento_a_agregar)
                i += max_longitud  # Ajustar el índice según la longitud del alimento encontrado más largo
            else:
                i += 1
        elementos_string = ', '.join(elementos_encontrados)
        nombre_columna='NERC '+tipo_entidad
        df.loc[k,nombre_columna]=elementos_string
    return df


In [47]:
vocab_alimentos_path='./../archivos/vocab_alimentos.json'
vocab_farmacos_path='./../archivos/vocab_farmacos_y_productos_quimicos.json'
vocab_sintomas_path='./../archivos/vocab_sintomas.json'
vocab_pruebas_clinicas_path='./../archivos/vocab_pruebas_clinicas.json'
# Cargamos el VOCABULARIO DE LOS ALIMENTOS
with open(vocab_alimentos_path, 'r') as archivo:
    vocab_alimentos = json.load(archivo)
with open(vocab_farmacos_path, 'r') as archivo:
    vocab_farmacos = json.load(archivo)
with open(vocab_sintomas_path, 'r') as archivo:
    vocab_sintomas = json.load(archivo)
with open(vocab_pruebas_clinicas_path, 'r') as archivo:
    vocab_pruebas_clinicas = json.load(archivo)

In [50]:

todos_los_elementos = [elemento['nombre'] for sublist in vocab_pruebas_clinicas.values() for elemento in sublist]
todos_los_elementos

['Hemoglobina A1c',
 'Glucosa en ayunas',
 'Prueba de tolerancia a la glucosa oral (PTGO)',
 'Test de cetonas en sangre',
 'Test de cetonas en orina',
 'Examen de microalbuminuria',
 'Perfil de lípidos',
 'Presión arterial',
 'Examen de fondo de ojo',
 'Prueba de función renal',
 'Monitoreo continuo de glucosa (MCG)',
 'Prueba de pie diabético',
 'Electrocardiograma (ECG)',
 'Velocidad de sedimentación globular (VSG)',
 'Prueba de tiroides',
 'Hemograma completo',
 'Prueba de vitamina B12',
 'Prueba de vitamina D',
 'Prueba de función hepática']